In [1]:
import os
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [2]:
import sys
sys.path.append('..\early-stopping-pytorch')
from pytorchtools import EarlyStopping

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
class AutoEncoder(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.input_shape = kwargs["input_shape"]
        # number of hidden units in first hidden layer
        self.n_units = kwargs["n_units"]
        self.latent_units = kwargs["latent_units"]
        
        self.encoder = torch.nn.Sequential(
            # Linear(): Initiate a linear function theta*x + b
            nn.Linear(in_features=self.input_shape, out_features=self.n_units),
            torch.nn.ReLU(),
            nn.Linear(in_features=self.n_units, out_features=self.latent_units),
            torch.nn.ReLU()
        )
        self.decoder = torch.nn.Sequential(
            nn.Linear(in_features=self.latent_units, out_features=self.n_units),
            torch.nn.ReLU(),
            nn.Linear(in_features=self.n_units, out_features=self.input_shape),
            torch.nn.Sigmoid()
        )
    
    # X denotes features
    def forward(self, X):
        encode = self.encoder(X)
        decode = self.decoder(encode)
        return decode

In [4]:
class SparseAutoEncoder(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.input_shape = kwargs["input_shape"]
        # number of hidden units in first hidden layer
        self.n_units = kwargs["n_units"]
        # number of hidden units in latent space
        self.latent_units = kwargs["latent_units"]
        
        self.encoder = torch.nn.Sequential(
            nn.Linear(in_features=self.input_shape, out_features=self.n_units),
            torch.nn.ReLU()
        )
        # Bottleneck is actually in the encoder, but it must be isolated in order to calculate sparsity
        self.bottleneck = torch.nn.Sequential(
            nn.Linear(in_features=self.n_units, out_features=self.latent_units),
            torch.nn.ReLU()
        )
        self.decoder = torch.nn.Sequential(
            nn.Linear(in_features=self.latent_units, out_features=self.n_units),
            torch.nn.ReLU(),
            nn.Linear(in_features=self.n_units, out_features=self.input_shape),
            torch.nn.Sigmoid()
        )
    
    # X denotes features
    def forward(self, X):
        encoded = self.encoder(X)
        bottleneck = self.bottleneck(encoded)
        decoded = self.decoder(bottleneck)
        return bottleneck, decoded

In [5]:
# 10,000 samples, 30x30 matrices
is_pca = False
data = np.ndarray(shape=(10000,30,30))
n_features = data.shape[1] * data.shape[2]

for i in range(10000):
    path = f'data/jet_matrices/sample{i+1}.dat'
    sample = np.loadtxt(path, unpack = True)
    data[i] = sample

print("Done loading data.")

Done loading data.


In [6]:
# Flatten data and convert to Torch Tensor

# 10,000 samples, 900 features
X = np.ndarray(shape=(10000, n_features))
for i, sample in enumerate(data):
    flat = sample.flatten()
    X[i] = flat
    #print(X[i])

X = torch.from_numpy(X)
# Convert all scalars to floats. May affect training behavior (ie. reconstructions made of non-binary scalar values)
X = X.float()
print(X)

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])


## PCA

In [ ]:
def de_correlate_data(X):
    X_pert = np.copy(X)
    i = 0
    for col in X.T:
        #print(col)
        X_pert[:,i] = np.random.permutation(col)
        #print(X_pert[:,i])
        i += 1
        
    return X_pert

# # function demo
# z = np.array([[1,2,3],[4,5,6],[7,8,9]])
# #z = np.array([[0,1,0],[1,0,1],[1,1,1]])
# print(z)
# X_pert = de_correlate_data(z)
# print(X_pert)

In [ ]:
# Plot cumulative explained variance w.r.t. number of components

def pca_run(X):
    pca = PCA(n_components=0.95).fit(X)

    #% matplotlib inline
    import matplotlib.pyplot as plt
    plt.rcParams["figure.figsize"] = (12,6)

    fig, ax = plt.subplots()
    y = np.cumsum(pca.explained_variance_ratio_)
    # n_components = number of components needed to reach cum. variance threshold
    n_components = y.size
    xi = np.arange(1, n_components+1, step=1)

    plt.ylim(0.0,1.1)
    plt.plot(xi, y, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    #change from 0-based array index to 1-based human-readable label
    plt.xticks(np.arange(0, n_components+1, step=1))
    plt.ylabel('Cumulative variance (%)')
    plt.title('The Number of Components Needed to Explain Variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.axhline(y=0.8, color='g', linestyle='-')
    plt.axhline(y=0.9, color='b', linestyle='-')
    plt.text(0, 0.915, '95% cut-off threshold', color = 'red', fontsize=13)
    plt.text(24, 0.85, '90% cut-off threshold', color = 'blue', fontsize=13)
    plt.text(12, 0.75, '80% cut-off threshold', color = 'green', fontsize=13)

    ax.grid(axis='x')
    plt.show()

# Run with original data.
pca_run(X.numpy())

# Run with permutated data.
# De-correlates features, so performing worse than original data indicates
# existence of correlation in the original data's features.
X_pert = de_correlate_data(X)
pca_run(X_pert)

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
fig, ax = plt.subplots()
plt.bar(xi, pca.explained_variance_ratio_, width=0.4)
plt.ylabel("Percent of Total Variance")
plt.xlabel("Principal Component")
plt.title("Significance of Each Principal Component Towards Variance ")

In [ ]:
# PCA

# Toggle to indicate to training that PCA is in use
is_pca = True
# -- DEFINE NUMBER OF COMPONENTS HERE --
n_components = 5

pca = PCA(n_components=n_components).fit(X.numpy())

print(X)
# If fails, re-run "Flatten data..." cell
X_pca = pca.fit_transform(X)
X_pca = torch.from_numpy(X_pca)
# Convert all scalars to floats. May affect training behavior (ie. reconstructions made of non-binary scalar values)
X_pca = X_pca.float()
# Replace former n_features with number of components
n_features = X_pca.shape[1]

## Training & Validation

In [7]:
# Hyperparameters

# Changes X based on whether PCA was used
if is_pca:
    X_2 = X_pca
else:
    X_2 = X

batch_size = 32
train_size = int(0.8 * len(X_2))
val_size = len(X_2) - train_size

In [8]:
# Initate data loaders

train, val = torch.utils.data.random_split(X_2, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=0
)

# Use gpu if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
device = torch.device("cuda")

In [ ]:
# # USE FOR MNIST ONLY

# input_shape = 784
# # Convert numpy array to tensor
# transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# # Define 
# train = torchvision.datasets.MNIST(
#     root="~/torch_datasets", train=True, transform=transform, download=True
# )

# test = torchvision.datasets.MNIST(
#     root="~/torch_datasets", train=False, transform=transform, download=True
# )

# train_loader = torch.utils.data.DataLoader(
#     train, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
# )

# test_loader = torch.utils.data.DataLoader(
#     test, batch_size=batch_size, shuffle=False, num_workers=4
# )

In [13]:
#############################    
#   TRAINING & VALIDATION   #
#############################

class NegativeValueError(Exception):
    """Raised when negative values are found in BCE loss"""
    pass

def kl_divergence(p, q):
    p = torch.nn.functional.softmax(p, dim=1)
    q = torch.nn.functional.softmax(q, dim=1)

    s1 = torch.sum(p * torch.log(p / q))
    s2 = torch.sum((1 - p) * torch.log((1 - p) / (1 - q)))
    s = s1 + s2
    return s

# Training and Validation are combined in order to allow for early stopping
def train_validate(model, epochs, lr, is_early_stopping=False, is_pca=False, is_sparse=False, patience=None, beta=None, rho=None):
    # Define Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Binary Cross Entropy Loss
    criterion = nn.BCELoss()

    # Reset model state if previously trained
    torch.manual_seed(1)
    def weights_init(m):
        if isinstance(m, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            print("existing instance")

    model.apply(weights_init)

    # Toggle Early Stopping (if using).
    if is_early_stopping:
        early_stopping = EarlyStopping(patience=patience, verbose=True)
        print("Using Early Stopping")
    if is_pca:
        print("Using PCA")

    print("Training...")

    for epoch in range(epochs):
        
        #############################    
        #          TRAINING         #
        #############################
        
        loss = 0
        # Prepare model for training
        model.eval()
        train_losses = []
        for i, batch in enumerate(train_loader, 0):

            # reshape mini-batch data from [batch_size, 30, 30] to [batch_size, 900]
            # load it to the active device
            batch = batch
            batch = batch.view(-1, n_features).to(device)

            # reset the gradients back to zero
            # PyTorch accumulates gradients on subsequent backward passes
            optimizer.zero_grad()

            # compute reconstructions
            # also retrieve bottleneck weights for computing sparsity penalty
            bottleneck, decoded = model(batch)

            # Exception handler for when BCE loss has negative values
            try:
                # compute training reconstruction loss
                train_loss = criterion(decoded, batch)

                if train_loss.item() < 0:
                    raise NegativeValueError
            except NegativeValueError:
                print("Negative values found in BCE loss (range should be [0.0, 1.0]")
                print(i)
                print(outputs)
                print(batch)
            
            # add sparsity penalty, if toggled
            if is_sparse:
                rho_hat = torch.sum(bottleneck, dim=0, keepdim=True)
                print(bottleneck.size())
                print(bottleneck)
                print(rho_hat)
                sparsity_penalty = beta * kl_divergence(rho, rho_hat)
                train_loss = train_loss + sparsity_penalty

            # compute accumulated gradients
            train_loss.backward()

            # perform parameter update based on current gradients
            optimizer.step()

            # add the mini-batch training loss to epoch loss
            train_losses.append(train_loss.item())

        # compute the epoch training loss
        avg_train_loss = np.average(train_losses)

        #############################    
        #         VALIDATION        #
        #############################

        # Decoupled into three lists due to issue with placing torch tensors into multidimensional lists
        batches = []
        recons = []
        val_losses = []

        # Prepare model for evaluation
        model.eval()

        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for i, batch in enumerate(val_loader, 0):
                batch = batch.view(-1, n_features).to(device)
                bottleneck, reconstructions = model(batch)
                # Reconstruction loss
                val_loss = criterion(reconstructions, batch)
                # Store samples, predictions, and loss for visualization purposes
                batches.append(batch)
                recons.append(reconstructions)
                val_losses.append(val_loss.item())
                #print(f'Batch {i}: {val_loss.item()}')

        avg_val_loss = np.average(val_losses)
        
        # display the epoch training loss and validation loss
        print("Epoch : {}/{}, Training Loss = {:.6f}, Validation Loss = {:.6f}".format(epoch + 1, epochs, avg_train_loss, avg_val_loss))
        
        if is_early_stopping:
            early_stopping(avg_val_loss, model)
            if early_stopping.early_stop:
                opt_epochs = epoch + 1
                print("Early stopping...")
                # Exit training loop
                break
    
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
    
    print(f"Epochs: {opt_epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")
    
    return  model, avg_train_loss, avg_val_loss

In [ ]:
# BASIC AUTOENCODER Execute training & validating

lr = 1e-3
epochs = 200
# number of hidden units in encoder hidden layer
n_units = 50
# number of hidden units in latent space
latent_units = 4
# Boolean for whether to use Early Stopping
is_early_stopping = False
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20

basic_model = SparseAutoEncoder(input_shape=n_features,
                    n_units=n_units,
                    latent_units=latent_units
                   ).to(device)

basic_model, avg_train_loss, avg_val_loss, opt_epochs = train_validate(model=basic_model,
                                                                        epochs=epochs,
                                                                        lr=lr,
                                                                        is_early_stopping=is_early_stopping, 
                                                                        is_pca=is_pca,
                                                                        patience=patience)


In [14]:
# SPARSE AUTOENCODER Execute training & validating

lr = 1e-3
epochs = 200
n_units = 50
latent_units = 10
is_early_stopping = False
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20

is_sparse = True
beta = 1
rho = 0.05
rho_tensor = torch.FloatTensor([rho for _ in range(latent_units)]).unsqueeze(0)

sparse_model = SparseAutoEncoder(input_shape=n_features,
                    n_units=n_units,
                    latent_units=latent_units
                   ).to(device)

sparse_model, avg_train_loss, avg_val_loss, opt_epochs = train_validate(model=sparse_model,
                                                    epochs=epochs,
                                                    lr=lr,
                                                    is_early_stopping=is_early_stopping, 
                                                    is_pca=is_pca,
                                                    is_sparse=is_sparse,
                                                    patience=patience,
                                                    beta=beta,
                                                    rho=rho_tensor)


existing instance
existing instance
existing instance
existing instance
Training...
torch.Size([32, 10])
tensor([[3.5157e-01, 1.4691e-02, 3.3282e-02, 5.9060e-01, 4.9301e-01, 6.8609e-01,
         0.0000e+00, 0.0000e+00, 5.2435e-03, 1.2886e-01],
        [2.2428e-01, 0.0000e+00, 0.0000e+00, 3.6739e-01, 5.1517e-01, 8.1970e-01,
         0.0000e+00, 0.0000e+00, 1.1490e-01, 3.2935e-01],
        [2.8877e-01, 0.0000e+00, 0.0000e+00, 5.0449e-01, 2.4746e-01, 7.6341e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.2778e-01],
        [5.1852e-01, 0.0000e+00, 0.0000e+00, 5.9909e-01, 2.0574e-01, 9.1507e-01,
         0.0000e+00, 0.0000e+00, 1.4271e-01, 2.6323e-01],
        [1.6477e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0099e-01, 1.0911e+00,
         0.0000e+00, 3.5601e-01, 4.0999e-01, 2.6429e-01],
        [4.8899e-01, 0.0000e+00, 6.4241e-02, 6.0147e-01, 4.6822e-01, 7.2228e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 2.1237e-01],
        [4.4418e-01, 0.0000e+00, 0.0000e+00, 6.4809e-01, 5.27

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
# PATH = './cifar_net.pth'
# torch.save(net.state_dict(), PATH)

## Testing

In [ ]:
# Testing

# Decoupled into three lists due to issue with placing torch tensors into multidimensional lists
batches = []
recons = []
test_losses = []

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    i = 1
    for batch in test_loader:
        batch = batch.view(-1, n_features).to(device)
        reconstructions = model(batch)
        # Reconstruction loss
        test_loss = criterion(reconstructions, batch)
        # Store samples, predictions, and loss for visualization purposes
        batches.append(batch)
        recons.append(reconstructions)
        test_losses.append(test_loss)
        print(f'Batch {i}: {test_loss.item()}')
        i += 1

test_loss_avg = sum(test_losses) / len(test_losses)
print(f"Average Test Reconstruction Loss: {test_loss_avg}")

## Visualization

In [ ]:
# Visualization

for i in range(len(recons)):
    loss = test_losses[i]
    batch = batches[i]
    reconstructions = recons[i]
    # Iterate through all examples in ith batch
    for j in range(len(batch)):
        # Reshape original example for plotting back into 30x30
        # or keep as vector of components if using PCA.
        if is_pca:
            original = batch[j].reshape(1, n_features)
        else:
            original = batch[j].reshape(data.shape[1], data.shape[2])
        original = original.cpu()
        # Reshape reconstructed example for plotting
        # or keep as vector of components if using PCA.
        if is_pca:
            reconstruction = reconstructions[j].reshape(1, n_features)
        else:
            reconstruction = reconstructions[j].reshape(data.shape[1], data.shape[2])
        reconstruction = reconstruction.cpu()
        
        fig = plt.figure(figsize=(8, 8))
        plt.title("Batch : {}/{}, Batch Reconstruction Loss = {:.6f}".format(i+1, len(recons), loss))
        plt.axis('off')
        # display original
        fig.add_subplot(1, 2, 1)
        plt.imshow(original)
        plt.axis('off')
        plt.title("original")
        plt.gray()
        
        # fig.get_xaxis().set_visible(False)
        # fig.get_yaxis().set_visible(False)

        # display reconstruction
        fig.add_subplot(1, 2, 2)
        plt.imshow(reconstruction)
        plt.axis('off')
        plt.title("reconstructed")
        plt.gray()
        # fig.get_xaxis().set_visible(False)
        # fig.get_yaxis().set_visible(False)
        plt.show()